<a href="https://colab.research.google.com/github/mitakshra/NlpEngineer/blob/Machine-Translation/NLP_ITV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
strr = "HI how are you weather is looking cool"

In [ ]:
import numpy as np
l = [[1,2,3],[2,3,4]]
print(np.mean(l))

In [ ]:
tokens = strr.split()

In [ ]:
tokens

In [ ]:
import numpy as np
x = np.random.randn(10,20)

In [ ]:
# Tokenization using tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
examples = ['We are learning natural language processing','we want to become expert of text processing',
            'Baba baba black sheep','welcome to word of data']

# build a tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(examples)


In [ ]:
# the dictionary of the examples has been created
word_idxs = tokenizer.word_index
print(word_idxs)

In [ ]:
word_indexes = [[1,6,7,8,9,2],[]]

In [ ]:
# vectors of the text
encoded_seq = tokenizer.texts_to_sequences(examples)

In [ ]:
encoded_seq

In [ ]:
padded_seq1 = pad_sequences(encoded_seq,padding='pre')

In [ ]:
padded_seq1

In [ ]:
padded_seq2 = pad_sequences(encoded_seq,padding='post')

In [ ]:
padded_seq2

In [ ]:
sent = ['Machine Learning expert are good']

enocoded_new = tokenizer.texts_to_sequences(sent)

In [ ]:
enocoded_new

In [ ]:
# update my tokenizer for handling OOV token
tokenizer1 = Tokenizer(oov_token='<OOV>')

In [ ]:
# build vocabulary again for the whole corpus
tokenizer1.fit_on_texts(examples)

In [ ]:
# new dictionary will come with OOV token
word_idx_new = tokenizer1.word_index
print(word_idx_new)

In [ ]:
enocoded_new1 = tokenizer1.texts_to_sequences(sent)

In [ ]:
enocoded_new1

Natural Language Tool kit

In [ ]:
import nltk

In [ ]:
nltk.download('all')

In [ ]:
# take a sentence
sentence = ' I am Rituraj Dixit an NLP Engineer woking on Sentiment Analysis and Translation models'
tokens = nltk.word_tokenize(sentence)

In [ ]:
tokens

In [ ]:
# part of speech tagging
from nltk.tag import pos_tag
pos_tagg = pos_tag(tokens,tagset='universal')

In [ ]:
pos_tagg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Mannual Tokenization**

In [ ]:
path_txt = '/content/drive/MyDrive/NLP/metaphorph.txt'

In [ ]:
file = open(path_txt,'rt')
txt = file.read()
file.close()

In [ ]:
print(txt)

In [ ]:
# splitting the words on the basis of whitespaces
words = txt.split()
words[0:20]

In [ ]:
# better approach is to use the regular expression which will filter only those 
# owrds which will fit into the pattern
import re
words_reg = re.split(r'\W+',txt)
words_reg[0:20]


In [ ]:
len(words_reg)

In [ ]:
# Normalizing the text
words_norm = [word.lower() for word in words_reg]

In [ ]:
words_norm[0:20]

In [ ]:
len(words_norm)

In [ ]:
# tokenization using nltk
from nltk.tokenize import word_tokenize
tokens_nltk = word_tokenize(txt)

In [ ]:
tokens_nltk[0:100]

In [ ]:
len(tokens_nltk)

In [ ]:
# remove all tokens that are not alphabetic
words_nltk = [word for word in tokens_nltk if word.isalpha()]

In [ ]:
len(words_nltk)

In [ ]:
# Filtering out the stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
print(stop_words)

In [ ]:
print(len(stop_words))

In [ ]:
refined_tokens  = [word for word in words_nltk if word not in stop_words]

In [ ]:
# stemming 
from nltk.stem.porter import PorterStemmer
# stemming of words
porter = PorterStemmer()
stemmed_words  = [porter.stem(word) for word in refined_tokens]

In [ ]:
stemmed_words[0:20]

**How to Prepare the Text Data using Scikit-learn**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text_doc = ["The quick brown fox jumped over the lazy dog"]
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text_doc)
# summarize the result
print(vectorizer.vocabulary_)

In [ ]:
# transform a single document
vector = vectorizer.transform(text_doc)

In [ ]:
vector.toarray()

In [ ]:
corpus = ["when Gregor Samsa woke from troubled dreams",
          "he found himself transformed in his bed into a horrible vermin",
          "The bedding was hardly able to cover it and seemed ready to slide off any moment.",
          "His many legs, pitifully thin compared with the size of the rest of him, waved about helplessly as he looked."]

In [ ]:
vectorizer1 = CountVectorizer()
vectors = [vectorizer1.fit_transform(c) for c in corpus]

In [ ]:
for v in vectors:
  print(v.toarray())

In [ ]:
# how to use tfids
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec  =TfidfVectorizer()
# vocabulary first
vocab_tfidf = tfidf_vec.fit(corpus)

In [ ]:
# print the vocabulary
print(vocab_tfidf.vocabulary_)

In [ ]:
vector1  = tfidf_vec.transform([corpus[0]])

In [ ]:
print(vector1.toarray())

The Bag-of-Words Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_imdb = '/content/drive/MyDrive/NLP/imdb_dataset/Train.csv'

In [ ]:
import pandas as pd
import numpy as np
import re
import string 
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv1D,Dense,AveragePooling1D,Flatten,Embedding
from tensorflow.keras.models import Sequential
from keras.utils import plot_model
from tensorflow.keras.models import load_model
from collections import Counter

In [ ]:
df_imdb = pd.read_csv(path_imdb)

In [ ]:
df_imdb.head()

In [ ]:
df_imdb

In [ ]:
def load_doc(path):
  file_1 = open(path,'rt')
  text = file_1.read()
  file_1.close()
  return text

In [ ]:
# 0: negative comments
# 1: Postive comments
# clean the data
def clean_doc(doc):
  # split the tokens using white spaces
  tokens = re.split(r'\W+',doc)
  # prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  # removing puctuation form each word
  tokens = [re_punc.sub('',word) for word in tokens]
  #remove tokens which are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
  # filter stop words
  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if token not in stop_words]
  # filter short words
  tokens = [token for token in tokens if len(token)>2]
  return tokens

In [ ]:
def add_doc_to_vocab(doc,vocab):
  # load to clean the doc
  tokens = clean_doc(doc)
  # clean tokens are get updated to the vocabularly 
  vocab.update(tokens)
  return vocab


In [ ]:
def save_tokens(tokens,filename):
  data = '\n'.join(tokens)
  # open a file
  vocab_file = open(filename,'w')
  # write the tokens in the file
  vocab_file.write(data)
  # cloase file
  vocab_file.close()

In [ ]:
# metamorphsis 2
path_sample = '/content/drive/MyDrive/NLP/Metaphorphis2.txt'

In [ ]:
sample_txt = load_doc(path_sample)

In [ ]:
tokens_meta = clean_doc(sample_txt)

In [ ]:
tokens_meta

In [ ]:
vocab1 = Counter()

In [ ]:
arr = df_imdb.iloc[1:4,0]

In [ ]:
arr

In [ ]:
vocab1 = Counter()

In [ ]:
for doc in arr.values:
  add_doc_to_vocab(sample_txt,vocab1)

In [ ]:
vocab1

In [ ]:
# minimum occurence=3
min_occurence = 3
filter_vocab_wrds = [w for w,c in vocab1.items() if c>=min_occurence]

In [ ]:
# save the filtered words in vocabulary in seprate text file
path_vocab = '/content/drive/MyDrive/NLP/Meta_vocab.txt'
save_tokens(filter_vocab_wrds,path_vocab)

#**Text prepration using Keras API**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.utils import plot_model
from sklearn.model_selection import train_test_split

In [ ]:
docs = [
    "Good work keep it up",
    "Very very good effort",
    "Nice work",
    "Excellent job",
    "enjoying NLP",
    "Learning new tranformers"

]

In [ ]:
# create an istance of Tokenizer
tk = Tokenizer()
# create the vocabulary
tk.fit_on_texts(docs)
# we use word_index to see the vocabulary
vocab = tk.word_index
print(vocab)

In [ ]:
# we have to convert these text elts into sequences
seq  = tk.texts_to_sequences(docs)

In [ ]:
len(vocab)

In [ ]:
seq

In [ ]:
seq1 = tk.texts_to_matrix(docs,mode='binary')

In [ ]:
seq1

In [ ]:
seq2 = tk.texts_to_matrix(docs,mode='count')

In [ ]:
seq2

In [ ]:
seq3 = tk.texts_to_matrix(docs,mode='tfidf')

In [ ]:
seq3

First Sentiment Analysis Model using Keras API

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path_imdb = '/content/drive/MyDrive/NLP/imdb_dataset/Train.csv'

In [ ]:
df_movie = pd.read_csv(path_imdb)

In [ ]:
X = df_movie['text']

In [ ]:
# vocabulary
tk_movie = Tokenizer()
tk_movie.fit_on_texts(X)

In [ ]:
vocab_movie = tk_movie.word_index

In [ ]:
print(vocab_movie)

In [ ]:
max_len = len(vocab_movie)
print(max_len)

In [ ]:
# we will create the matrices for all the three modes and then we will analyze the accuracy
seq_count = tk_movie.texts_to_matrix(X,mode='count')

In [ ]:
seq_tfidf= tk_movie.texts_to_matrix(X,mode="tfidf")

In [ ]:
seq_bin = tk_movie.texts_to_matrix(X,mode='binary')

In [ ]:
Y = df_movie['label']

In [ ]:
train_x,text_x,train_y,test_y = train_test_split(seq_count,Y,test_size=0.3)

In [ ]:
# defining the model
model_movie = Sequential()
model_movie.add(Dense(32,input_shape=(max_len+1,),activation='tanh'))
model_movie.add(Dense(1,activation='sigmoid'))
model_movie.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model_movie.summary()

In [ ]:
model_movie.fit(train_x,train_y,epochs=10)

In [ ]:
train_x_tf,text_x_tf,train_y_tf,test_y_tf = train_test_split(seq_tfidf,Y,test_size=0.3)

In [ ]:
model_movie.fit(train_x,train_y,epochs=10)

#The Word Embedding Models

In [ ]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA 
from matplotlib import pyplot

In [ ]:
# define a training data
sentences = [['this','is','a','sentence','of','word2vec'],
             ['this', 'is','a','second','sentence', 'of','word2vec'],
             ['now', 'we','are','learning','word','embedding', 'for','NLP'],
             ['and','this','is','the','final','sentence'],
             ['the','word','embedding','is','used','to','grasp','semantic','information']]

In [ ]:
# train model 
model_w2v = Word2Vec(sentences,min_count=1)

In [ ]:
# summarize the model
print(model_w2v)

In [ ]:
# summarize the vocabulary
words = list(model_w2v.wv.vocab)
print(words)

In [ ]:
# acess a word vector
print(len(model_w2v['to']))

In [ ]:
model_w2v['to']

In [ ]:
# visulaizing the model
X = model_w2v[model_w2v.wv.vocab]

In [ ]:
print(X)

In [ ]:
X.shape

In [ ]:
X1 = X[:,0:2]

In [ ]:
X1

In [ ]:
# principal component analysis is the effective technique to reduce the dimensions of the data 
# without loosing the meaning
pca = PCA(n_components=2)
reduced_mtx = pca.fit_transform(X)

In [ ]:
reduced_mtx

In [ ]:
reduced_mtx.shape

In [ ]:
pyplot.scatter(reduced_mtx[:,0],reduced_mtx[:,1])

In [ ]:
words  = list(model_w2v.wv.vocab)

In [ ]:
for i,word in enumerate(words):
  pyplot.annotate(word,xy=(reduced_mtx[i,0],reduced_mtx[i,1]))
pyplot.show()

# Sentiment Analysis using financial sentiment analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.decomposition import PCA 
from matplotlib import pyplot
import string
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.layers import Conv1D,Dense,AveragePooling1D,Flatten,Embedding
from tensorflow.keras.models import Sequential
from keras.utils import plot_model,pad_sequences
from tensorflow.keras.models import load_model
from collections import Counter

In [ ]:
nltk.download('all')

In [ ]:
path_financial = '/content/drive/MyDrive/NLP/financial_sentiment.csv'

In [ ]:
df_financial = pd.read_csv(path_financial)

In [ ]:
df_financial.shape

In [ ]:
df1 = df_financial.sample(frac=1).reset_index(drop=True)

In [ ]:
df1.head()

In [ ]:
df1['Sentiment'].value_counts()

In [ ]:
def clean_doc(doc):
  # split the tokens using white spaces
  tokens = re.split(r'\W+',doc)
  # prepare regex for char filtering
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  # removing puctuation form each word
  tokens = [re_punc.sub('',word) for word in tokens]
  #remove tokens which are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
  # filter stop words
  stop_words = set(stopwords.words('english'))
  tokens = [token for token in tokens if token not in stop_words]
  # filter short words
  tokens = [token for token in tokens if len(token)>=2]
  return tokens

In [ ]:
X = df1['Sentence']

In [ ]:
X[4106]

In [ ]:
sentence_tokens = [clean_doc(sent) for sent in X ]

In [ ]:
sentence_tokens

In [ ]:
# train the sentiment model 
financial_w2v = Word2Vec(sentence_tokens,min_count=1)

In [ ]:
# summarize the vocabulary
financial_words = list(financial_w2v.wv.vocab)
print(financial_words)

In [ ]:
v

In [ ]:
X_data = []
index=1
for tkn_lst in sentence_tokens:
  lst = []
  for word in tkn_lst:
    lst.append(financial_w2v[word])
  if len(lst)==0:
    print(index)
  X_data.append(lst)
  index=index+1


In [ ]:
sentence_tokens[4107]

In [ ]:
sentence_tokens[4106]

In [ ]:
X_data_flatten = []
for vectors in X_data:
  if len(vectors)>=1:
    X_data_flatten.append(list(np.concatenate(vectors).flat))

In [ ]:
len(X_data_flatten)

In [ ]:
Y = df1['Sentiment'].map({'neutral':0,'positive':1,'negative':2})

In [ ]:
type(Y)

In [ ]:
Y = Y.drop(Y.index[4106])

In [ ]:
len(Y)

In [ ]:
max_length = max([len(vector) for vector in X_data_flatten])

In [ ]:
max_length

In [ ]:
min_length = min([len(vector) for vector in X_data_flatten])

In [ ]:
min_length

In [ ]:
final_vector = []

In [ ]:
for vector in X_data_flatten:
  diff = max_length-len(vector)
  for val in range(diff):
    vector.append(0)
  final_vector.append(vector)


In [ ]:
min_length = min([len(vector) for vector in final_vector])

In [ ]:
min_length

In [ ]:
# Now we build the autoencoder for dimensionality reduction


# Implemeting Financial Setiment using tensorflow

In [ ]:
tk_fin = Tokenizer()

In [ ]:
tk_fin.fit_on_texts(df1['Sentence'])

In [ ]:
# vocabulary of financial setiments
vocab_financial = tk_fin.word_index

In [ ]:
# we can create sequences
X_seq = tk_fin.texts_to_sequences(df1['Sentence'])

In [ ]:
# padding
# max size
max_len = max([len(x) for x in X_seq])

In [ ]:
X_padded = pad_sequences(X_seq,padding='post')

In [ ]:
# create a model
model_fin = Sequential()
model_fin.add(Embedding(len(vocab_financial)+1,100,input_length=max_len))
model_fin.add(Conv1D(32,5,activation='tanh'))
model_fin.add(AveragePooling1D(2))
model_fin.add(Flatten())
model_fin.add(Dense(32,activation='tanh'))
model_fin.add(Dense(3,activation='softmax'))
model_fin.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model_fin.summary()

In [ ]:
plot_model(model_fin)

In [ ]:
Y = df1['Sentiment'].map({'neutral':0,'positive':1,'negative':2})

In [ ]:
Y

In [ ]:
# split the data
train_x,test_x,train_y,test_y = train_test_split(X_padded,Y,test_size=0.3)

In [ ]:
# fit the model
history_fin = model_fin.fit(train_x,train_y,epochs=10)

#Sequence Models

**Recurrent Neural Networks**

In [ ]:
import numpy as np


In [ ]:
a_0 = np.random.randn(4)

In [ ]:
a_0

In [ ]:
W_aa = np.random.randn(4,4)

In [ ]:
W_aa

In [ ]:
x_1 = np.random.randn(6)

In [ ]:
W_xa = np.random.randn(6,4)

In [ ]:
b_a = np.random.rand(4)

In [ ]:
a_1 = np.tanh(np.dot(a_0,W_aa)+ np.dot(x_1,W_xa) + b_a)

In [ ]:
a_1

In [ ]:
# Weight martix for output and let us suppose we have only three classes
# PRN, NN, VB
W_ay = np.random.rand(4,3)
b_y = np.random.rand(3)

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [ ]:
y_val = np.dot(a_1,W_ay)+b_y

In [ ]:
y_val

In [ ]:
scores = softmax(y_val)

In [ ]:
scores

# RNN for Financial Sentiment Analysis

In [ ]:
# the normalize data is there in X_padded
# create a RNN model
from tensorflow.keras.layers import SimpleRNN
embdedding_size = 32
vocab_size = len(vocab_financial)+1
# max size
max_len = max([len(x) for x in X_seq])



In [ ]:
model_rnn = Sequential()
model_rnn.add(Embedding(vocab_size,embdedding_size,input_length= max_len ))
model_rnn.add(SimpleRNN(64))
model_rnn.add(Dense(32,activation='tanh'))
model_rnn.add(Dense(1,activation='sigmoid'))
model_rnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model_rnn.summary()

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(X_padded,Y,test_size=0.3)

In [ ]:
# fit the mdoel
history = model_rnn.fit(train_x,train_y,epochs=10)

**GRU Cells (Gated Recurrent Units)**

In [ ]:
from tensorflow.keras.layers import GRU
import numpy as np


In [ ]:
# hidden state information
a_t0 = np.random.randn(5)
# weight matrices for relevance gate
W_aa_r = np.random.randn(5,5)
W_xa_r = np.random.randn(5,5)
#weight matrices for update gates
W_aa_u = np.random.randn(5,5)
W_xa_u = np.random.randn(5,5)
#weight matrices for c_tilde gates
W_aa_c = np.random.randn(5,5)
W_xa_c = np.random.randn(5,5)
# input information 
x_t = np.random.rand(5)
# bias initialization
b_r = np.random.rand(5)
b_u = np.random.rand(5)
b_c = np.random.rand(5)



In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [ ]:
# relevance gate value
lamda_r = np.dot(a_t0,W_aa_r)+np.dot(x_t,W_xa_r)+b_r

In [ ]:
lamda_r_sig = [sigmoid(val) for val in lamda_r]

In [ ]:
lamda_r_sig

In [ ]:
# update gate value
lamda_u = np.dot(a_t0,W_aa_u)+np.dot(x_t,W_xa_u)+b_u

In [ ]:
lamda_u_sig = [sigmoid(val) for val in lamda_u]

In [ ]:
lamda_u_sig

[0.9624778335224151,
 0.9936441097907869,
 0.9960408284440706,
 0.9721185397999369,
 0.9075891678526896]

TypeError: ignored

In [ ]:
# c_tilde
elt_mul = np.array([lamda_u_sig[i]*a_t0[i] for i in range(len(a_t0))])
c_tilde = np.tanh(np.dot(W_aa_c,elt_mul)+np.dot(x_t,W_xa_c)+b_c)

In [ ]:
c_t = 